In [1]:
import os
import sys
import datetime
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import muon as mu
import mudata as md
import scanpy as sc
from scipy.io import mmread

2025-08-14 00:52:33.877112: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755147153.892660  332810 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755147153.897396  332810 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755147153.909569  332810 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755147153.909583  332810 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755147153.909586  332810 computation_placer.cc:177] computation placer alr

In [2]:
mainDir = '/home/cegrad/shriramp/SCREAM_wf'
inputDir = os.path.join(mainDir, 'inputfiles')
rawDir = os.path.join(inputDir, 'raw')
processedDir = os.path.join(inputDir, 'processed')
resultsDir = os.path.join(mainDir, 'results')

In [3]:
meta = pd.read_csv(os.path.join(rawDir, 'processed', 'pbmc_multiomic_metadata.csv'), index_col=0)
lsi = pd.read_csv(os.path.join(rawDir, 'processed', 'pbmc_multiomic_lsi.csv'), index_col=0)
pca = pd.read_csv(os.path.join(rawDir, 'processed', 'pbmc_multiomic_pca.csv'), index_col=0)
var_feats = pd.read_csv(os.path.join(rawDir, 'processed', 'pbmc_multiomic_varfeats.csv'), index_col=0)

In [4]:
X_atac_counts = mmread(os.path.join(rawDir, 'processed', 'pbmc_multiomic_ataccounts.mtx')).tocsc()
X_atac_lsi = mmread(os.path.join(rawDir, 'processed', 'pbmc_multiomic_atacdata.mtx')).tocsc()
var_names = pd.read_csv(os.path.join(rawDir, 'processed', 'pbmc_multiomic_Seurat_peaks.tsv'), header=None)[0].values

adata_atac = sc.AnnData(X_atac_counts.T)
adata_atac.layers['raw'] = adata_atac.X.copy()
adata_atac.layers['lsi'] = X_atac_lsi.T
adata_atac.var_names = var_names

# adata_atac.obs = atac_meta.copy()
adata_atac.var['modality'] = 'peaks'
adata_atac.obs = meta.copy()

adata_atac.obsm['lsi'] = lsi.values.copy()

In [5]:
X_rna_counts = mmread(os.path.join(rawDir, 'processed', 'pbmc_multiomic_rnacounts.mtx')).tocsc()
X_rna_data = mmread(os.path.join(rawDir, 'processed', 'pbmc_multiomic_rnasct.mtx')).tocsc()
var_names = pd.read_csv(os.path.join(rawDir, 'processed', 'pbmc_multiomic_Seurat_genes.tsv'), header=None)[0].values

adata_rna = sc.AnnData(X_rna_counts.T)
adata_rna.layers['raw'] = adata_rna.X.copy()
adata_rna.layers['sct'] = X_rna_data.T
adata_rna.var_names = var_names

# adata_atac.obs = atac_meta.copy()
adata_rna.var['modality'] = 'genes'
adata_rna.obs = meta.copy()

adata_rna.obsm['pca'] = pca.values.copy()
adata_rna.uns['varfeats'] = var_feats['x'].tolist()

In [7]:
mdata = mu.MuData({'RNA': adata_rna, 'ATAC': adata_atac})

/home/cegrad/shriramp/miniconda3/envs/Shri/lib/python3.12/site-packages/mudata/_core/mudata.py:1598: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("var", axis=0, join_common=join_common)
/home/cegrad/shriramp/miniconda3/envs/Shri/lib/python3.12/site-packages/mudata/_core/mudata.py:1461: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("obs", axis=1, join_common=join_common)


In [9]:
adata_2 = sc.read_h5ad('/home/cegrad/shriramp/SCREAM_wf/inputfiles/raw/rna_atac_multi_latent_adata_sae_appr_kmean_new_non_scaled_no_bg2_lsi299_clipped10_new1_scream_hyperbs90.h5ad')

,cell_types_anno
0,
AAACAGCCAAGGAATC-1,CD4 Naive
AAACAGCCAATCCCTT-1,CD4 TCM
AAACAGCCAATGCGCT-1,CD4 Naive
AAACAGCCACCAACCG-1,CD8 Naive
AAACAGCCAGGATAAC-1,CD4 Naive
...,...
TTTGTTGGTGACATGC-1,CD8 Naive
TTTGTTGGTGTTAAAC-1,CD8 Naive
TTTGTTGGTTAGGATT-1,NK


In [18]:
mdata.obs.columns

Index(['RNA:orig.ident', 'RNA:nCount_RNA', 'RNA:nFeature_RNA',
       'RNA:nCount_ATAC', 'RNA:nFeature_ATAC', 'RNA:nucleosome_signal',
       'RNA:nucleosome_percentile', 'RNA:TSS.enrichment', 'RNA:TSS.percentile',
       'RNA:percent.mt', 'RNA:nCount_SCT', 'RNA:nFeature_SCT',
       'RNA:predicted.id', 'RNA:prediction.score.CD14.Mono',
       'RNA:prediction.score.CD4.TCM', 'RNA:prediction.score.CD8.Naive',
       'RNA:prediction.score.NK', 'RNA:prediction.score.CD8.TEM',
       'RNA:prediction.score.CD16.Mono', 'RNA:prediction.score.B.intermediate',
       'RNA:prediction.score.CD4.Naive', 'RNA:prediction.score.CD4.CTL',
       'RNA:prediction.score.B.naive', 'RNA:prediction.score.MAIT',
       'RNA:prediction.score.gdT', 'RNA:prediction.score.CD8.TCM',
       'RNA:prediction.score.dnT', 'RNA:prediction.score.B.memory',
       'RNA:prediction.score.Doublet', 'RNA:prediction.score.pDC',
       'RNA:prediction.score.CD8.Proliferating', 'RNA:prediction.score.Treg',
       'RNA:predictio

In [22]:
mdata.obs = mdata.obs.join(adata_2.obs.set_index('0')[['cell_types_anno']], how='left')

In [24]:
mdata.write_h5mu(os.path.join(processedDir, '10XPBMC_processed.h5mu'))

/home/cegrad/shriramp/miniconda3/envs/Shri/lib/python3.12/site-packages/mudata/_core/mudata.py:1598: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("var", axis=0, join_common=join_common)
/home/cegrad/shriramp/miniconda3/envs/Shri/lib/python3.12/site-packages/mudata/_core/mudata.py:1461: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("obs", axis=1, join_common=join_common)
